## Smart RAG
- Know what you know and what you don't know.


In [1]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings


# Load doc
layzer = UpstageLayoutAnalysisLoader("solar_paper.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [5]:
# Tools
from langchain_core.tools import tool
import requests
import os
from tavily import TavilyClient
from langchain_upstage import ChatUpstage

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


@tool
def solar_paper_search(query: str) -> str:
    """Query for research paper about solarllm, dus, llm and general AI.
    If the query is about DUS, Upstage, AI related topics, use this.
    """
    return docs[0].page_content


@tool
def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google.
    Query for general topics.
    """
    return tavily.search(query=query)


@tool
def get_news(topic: str) -> str:
    """Get latest news about a topic.
    If users are more like recent news, use this.
    """
    # https://newsapi.org/v2/everything?q=tesla&from=2024-04-01&sortBy=publishedAt&apiKey=API_KEY
    # change this to request news from a real API
    news_url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={os.environ['NEWS_API_KEY']}"
    respnse = requests.get(news_url)
    return respnse.json()


tools = [solar_paper_search, internet_search, get_news]

llm = ChatUpstage()
llm_with_tools = llm.bind_tools(tools)

In [6]:
def call_tool(tool_call):
    tool_name = tool_call["name"].lower()
    if tool_name not in globals():
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [15]:
llm_with_tools.invoke("What is Solar LLM?").tool_calls


[{'name': 'solar_paper_search',
  'args': {'query': 'Solar LLM'},
  'id': 'c53b669b-3e90-4269-88f9-ec08aec82ec7'}]

In [16]:
llm_with_tools.invoke("What is news about Seoul").tool_calls


[{'name': 'get_news',
  'args': {'topic': 'Seoul'},
  'id': 'd73075e2-4de8-4582-848c-03525ed3b203'}]

In [17]:
llm_with_tools.invoke("What's best place in Seoul?").tool_calls


[{'name': 'internet_search',
  'args': {'query': 'Best place in Seoul'},
  'id': 'c5726e9e-9112-45d9-9183-8046d58e4609'}]

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [28]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def tool_rag(question):
    tool_calls = llm_with_tools.invoke(question).tool_calls
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool(tool_call))

    print(context)
    
    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [31]:
tool_rag("What is Solar llm?")

[]



"Solar LLC is a type of business structure that is commonly used in the solar energy industry. It is a limited liability company, which means that the owners, or members, of the company are not personally liable for the debts and liabilities of the company. This provides a layer of protection for the owners' personal assets. Solar LLCs are often formed to develop, own, and operate solar energy projects, such as solar farms or rooftop solar installations. By forming an LLC, the owners can potentially benefit from tax advantages and can also provide a clear legal structure for the ownership and operation of the solar energy project."

In [33]:
tool_rag("What is news about Tesla?")

[{'name': 'get_news', 'args': {'topic': 'Tesla'}, 'id': '78eb6996-9298-4850-9e21-daac8eb8484b'}]
{'status': 'ok', 'totalResults': 15741, 'articles': [{'source': {'id': 'wired', 'name': 'Wired'}, 'author': 'Brian Barrett', 'title': 'Tesla Recalls Cybertruck Over Trapped Pedals—Its Worst Flaw Yet', 'description': 'Tesla has issued a voluntary recall for nearly 4,000 Cybertrucks because the accelerator pad could get trapped, pushing the car to full speed.', 'url': 'https://www.wired.com/story/cybertruck-recall-accelerator-pad-tesla/', 'urlToImage': 'https://media.wired.com/photos/66227bca255c13bb362f533a/191:100/w_1280,c_limit/Cybertruck-Recall-Gear-GettyImages-2025755700.jpg', 'publishedAt': '2024-04-19T14:22:47Z', 'content': 'Teslas Cybertruck has been widely derided. Its panel gaps are wide and amateurish, its prone to rust, and it looks like an ergonomic cheese grater. Its most serious flaw to date, though, has resulted… [+3426 chars]'}, {'source': {'id': None, 'name': '[Removed]'}, '

'Tesla has issued a voluntary recall for nearly 4,000 Cybertrucks due to a faulty accelerator pad that could get trapped, causing the car to go to full speed. This is the most serious flaw for the Cybertruck to date. Additionally, Tesla has been in the news for other reasons, including layoffs, price cuts, and issues with its self-driving technology.'

In [34]:
tool_rag("iPhone 13 spec?")

[{'name': 'internet_search', 'args': {'query': 'iPhone 13 spec'}, 'id': 'f0b71104-daa5-465b-8719-e175918bb362'}]
None


'iPhone 13 specifications\n\nThe iPhone 13 is a smartphone developed by Apple Inc. It was announced on September 14, 2021, as part of the iPhone 13 series, which also includes the iPhone 13 Mini, iPhone 13 Pro, and iPhone 13 Pro Max. The iPhone 13 features a 6.1-inch Super Retina XDR OLED display with ProMotion technology, which supports a refresh rate of up to 120Hz. It is powered by the Apple A15 Bionic chip, which has a 6-core CPU and a 4-core GPU. The iPhone 13 has a dual-camera system with a 12MP wide-angle lens and a 12MP ultra-wide-angle lens, and it supports 4K video recording at up to 60fps. The device has 5G connectivity and is available in 128GB, 256GB, and 512GB storage options. It has a battery life of up to 19 hours of video playback and supports MagSafe charging. The iPhone 13 is available in five colors: pink, blue, midnight, starry white, and product red.'

# Excercise
Solar LLM is small, so it might not give the best results for complex tasks. For those, you can use larger LLMs like GPT-4. However, for quick answers and summaries, using a small size LLM like Solar can give better performance and efficiency.
